### Resolução das questões do arquivo `perguntas_sql.md`

#### Imports

In [1]:
import pandas as pd
import basedosdados as bd

In [2]:
query = "SELECT * FROM `datario.adm_central_atendimento_1746.chamado` LIMIT 10"
df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")
df.head()

Downloading: 100%|██████████| 10/10 [00:00<00:00, 31.97rows/s]


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,9383627,1,2017-02-15 18:55:19,2021-11-24 17:23:35,24,2,71084,72,1208,SEGOV,...,2021-11-24 17:23:35,<NA>,D,F,No prazo,Encerrado,Andamento,None,0,2017-02-01
1,9369509,1,2017-02-13 16:13:52,2018-07-30 16:50:11,20,2,77685,0,979,1ª SGCO/DARB,...,2018-07-30 16:50:11,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2017-02-01
2,9376683,1,2017-02-14 17:15:22,2018-09-19 12:49:16,24,2,83949,29,979,1ª SGCO/DARB,...,2018-09-19 12:49:16,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2017-02-01
3,9376655,1,2017-02-14 17:10:12,2018-09-19 12:28:25,24,2,77271,90,979,1ª SGCO/DARB,...,2018-09-19 12:28:25,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2017-02-01
4,9369524,1,2017-02-13 16:15:42,2018-07-30 16:43:41,20,2,112375,<NA>,979,1ª SGCO/DARB,...,2018-07-30 16:43:41,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2017-02-01


#### 1. Quantos chamados foram abertos no dia 01/04/2023?

In [3]:
query = """
SELECT COUNT(*) AS total_chamados 
FROM `datario.adm_central_atendimento_1746.chamado` 
WHERE DATE(data_inicio) = '2023-04-01';
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.27rows/s]

   total_chamados
0            1756


#### 2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?


In [4]:
query = """
SELECT tipo_situacao, COUNT(*) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado` 
WHERE date(data_inicio) = '2023-04-01'
GROUP BY tipo_situacao
ORDER BY total_chamados DESC
LIMIT 1;
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.14rows/s]

  tipo_situacao  total_chamados
0      Atendido            1015


#### 3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?


In [5]:
query = """
SELECT 
  b.nome AS bairro, 
  COUNT(id_chamado) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.dados_mestres.bairro` b ON c.id_bairro = b.id_bairro
WHERE DATE(c.data_inicio) = '2023-04-01'
GROUP BY b.nome
ORDER BY total_chamados DESC
LIMIT 03;
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 3/3 [00:00<00:00,  8.67rows/s]

            bairro  total_chamados
0     Campo Grande             113
1           Tijuca              89
2  Barra da Tijuca              59


#### 4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [6]:
query = """
SELECT 
  b.subprefeitura,
  COUNT(c.id_chamado) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.dados_mestres.bairro` b ON c.id_bairro = b.id_bairro
WHERE DATE(data_inicio) = '2023-04-01'
GROUP BY b.subprefeitura
ORDER BY total_chamados DESC
LIMIT 01;
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.49rows/s]

  subprefeitura  total_chamados
0    Zona Norte             510


#### 5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [7]:
query = """
SELECT
    COUNT(c.id_chamado) AS chamados_sem_associacao
FROM `datario.adm_central_atendimento_1746.chamado` c
LEFT JOIN `datario.dados_mestres.bairro` b ON c.id_bairro = b.id_bairro
WHERE DATE(data_inicio) = '2023-04-01'
AND (b.id_bairro IS NULL OR b.subprefeitura IS NULL);
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.46rows/s]

   chamados_sem_associacao
0                       73


In [8]:
query = """
SELECT
    c.id_chamado,
    c.id_bairro,
    b.nome,
    b.subprefeitura
FROM `datario.adm_central_atendimento_1746.chamado` c
LEFT JOIN `datario.dados_mestres.bairro` b ON c.id_bairro = b.id_bairro
WHERE DATE(data_inicio) = '2023-04-01'
AND (b.id_bairro IS NULL OR b.subprefeitura IS NULL);
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 73/73 [00:00<00:00, 248.76rows/s]

   id_chamado id_bairro  nome subprefeitura
0    18517061      None  None          None
1    18517502      None  None          None
2    18517263      None  None          None
3    18517485      None  None          None
4    18517022      None  None          None
..        ...       ...   ...           ...
68   18516523      None  None          None
69   18517483      None  None          None
70   18518366      None  None          None
71   18517314      None  None          None
72   18516414      None  None          None

[73 rows x 4 columns]


#### 6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [9]:
query = """
SELECT
   COUNT(id_chamado) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE subtipo = 'Perturbação do sossego'
AND DATE(data_inicio) BETWEEN '2022-01-01' AND '2023-12-31';
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.26rows/s]

   total_chamados
0           42830


#### 7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).


In [10]:
query = """
SELECT
  c.*
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` o 
  ON DATE(c.data_inicio) BETWEEN DATE(o.data_inicial) AND DATE(o.data_final)
WHERE subtipo = 'Perturbação do sossego';
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 1214/1214 [00:00<00:00, 1422.32rows/s]

     id_chamado id_origem_ocorrencia         data_inicio            data_fim  \
0      17661698                   11 2022-09-02 22:40:02 2022-09-09 09:12:54   
1      17682627                   11 2022-09-10 00:55:05 2022-09-15 05:16:25   
2      17664662                   13 2022-09-04 22:26:55 2022-09-04 23:08:19   
3      17674558                    1 2022-09-08 00:07:31 2022-09-08 05:47:39   
4      17675616                   22 2022-09-08 10:41:03 2022-09-15 17:23:00   
...         ...                  ...                 ...                 ...   
1209   18080238                   11 2023-01-01 20:01:27 2023-01-11 08:26:36   
1210   18078387                    1 2022-12-30 23:22:36 2022-12-31 06:14:05   
1211   18077790                    1 2022-12-30 17:00:19 2023-01-10 09:37:50   
1212   18075893                   11 2022-12-30 00:13:57 2023-01-10 09:22:16   
1213   18079154                   11 2022-12-31 14:03:28 2023-01-10 10:17:52   

     id_bairro id_territorialidade id_l

- Como a tabela de Eventos só possui esses três eventos não preciso especificar na query, mas caso houvesse outros eventos eu adicionaria este filtro.
- Join entre as tabelas para considerar qualquer chamado que ocorreu durante o evento (independente de ser início, no meio ou no final do evento)

#### 8. Quantos chamados desse subtipo foram abertos em cada evento?


In [11]:
query = """
SELECT
  o.evento,
  COUNT(c.id_chamado) AS total_chamados
FROM `datario.adm_central_atendimento_1746.chamado` c
JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` o 
  ON DATE(c.data_inicio) BETWEEN DATE(o.data_inicial) AND DATE(o.data_final)
WHERE subtipo = 'Perturbação do sossego'
GROUP BY o.evento
ORDER BY total_chamados DESC;
"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)

Downloading: 100%|██████████| 3/3 [00:00<00:00, 10.48rows/s]

        evento  total_chamados
0  Rock in Rio             834
1     Carnaval             241
2    Reveillon             139


- Como a tabela de Eventos só possui esses três eventos não preciso especificar na query, mas caso houvesse outros eventos eu adicionaria este filtro.
- Join entre as tabelas para considerar qualquer chamado que ocorreu durante o evento (independente de ser início, no meio ou no final do evento)

#### 9. Qual evento teve a maior média diária de chamados abertos desse subtipo?


In [12]:
query = """
WITH chamados_por_dia AS (
  SELECT 
    o.evento AS evento, 
    DATE(c.data_inicio) AS dia, 
    COUNT(c.id_chamado) AS total_chamados
  FROM `datario.adm_central_atendimento_1746.chamado` c
  JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` o 
    ON DATE(c.data_inicio) BETWEEN DATE(o.data_inicial) AND DATE(o.data_final)
  WHERE c.subtipo = 'Perturbação do sossego'
  GROUP BY o.evento, DATE(c.data_inicio)
  )

SELECT 
  evento,
  ROUND(AVG(total_chamados), 2) AS media_diaria
FROM chamados_por_dia
GROUP BY evento
ORDER BY media_diaria DESC
LIMIT 1;

"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.37rows/s]

        evento  media_diaria
0  Rock in Rio        119.14


#### 10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [13]:
query = """
WITH media_chamados_evento AS (
  SELECT 
    o.evento AS evento, 
    DATE(c.data_inicio) AS dia, 
    COUNT(c.id_chamado) AS total_chamados
  FROM `datario.adm_central_atendimento_1746.chamado` c
  JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` o 
    ON DATE(c.data_inicio) BETWEEN DATE(o.data_inicial) AND DATE(o.data_final)
  WHERE c.subtipo = 'Perturbação do sossego'
  GROUP BY o.evento, DATE(c.data_inicio)
  ),

media_chamados_periodo AS (
  SELECT 
    DATE(c.data_inicio) AS dia, 
    COUNT(c.id_chamado) AS total_chamados
  FROM `datario.adm_central_atendimento_1746.chamado` c
  WHERE c.subtipo = 'Perturbação do sossego'
  AND DATE(c.data_inicio) BETWEEN '2022-01-01' AND '2023-12-31'
  GROUP BY DATE(c.data_inicio)
)

SELECT 
  'Média durante os eventos' AS descricao,
  ROUND(AVG(mce.total_chamados), 2) AS media_diaria_evento
FROM media_chamados_evento mce

UNION ALL

SELECT 
  'Média geral do período' AS descricao,
  ROUND(AVG(mcp.total_chamados), 2) AS media_diaria_total
FROM media_chamados_periodo mcp;


"""


df = bd.read_sql(query, billing_project_id="projeto-prefeitura-rio")

print(df)


Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.52rows/s]

                  descricao  media_diaria_evento
0    Média geral do período                61.98
1  Média durante os eventos                86.71
